In [ ]:
from micro_namedtuple_sqlite_persister.persister import Engine
from micro_namedtuple_sqlite_persister.query2 import select, select_query
from typing import NamedTuple

In [ ]:
class League(NamedTuple):
    id: int | None
    leaguename: str

class Team(NamedTuple):
    id: int | None
    teamname: str
    league: League

class Athlete(NamedTuple):
    id: int | None
    name: str
    team: Team


engine = Engine(":memory:")
engine.ensure_table_created(League, force_recreate=True)
engine.ensure_table_created(Team, force_recreate=True)
engine.ensure_table_created(Athlete, force_recreate=True)

# Generate dummy data
leagues = [engine.save(League(None, "Big")), engine.save(League(None, "Small"))]
teams = [
    engine.save(Team(None, "Red", leagues[0])),
      engine.save(Team(None, "Ramble", leagues[1])),
      engine.save(Team(None, "Blue", leagues[0])),
      engine.save(Team(None, "Green", leagues[1])),
      ]
players = [
    engine.save(Athlete(None, "Alice", teams[0])),
    engine.save(Athlete(None, "Bob", teams[0])),
    engine.save(Athlete(None, "Charlie", teams[1])),
    engine.save(Athlete(None, "Dave", teams[2])),
    engine.save(Athlete(None, "Beth", teams[3])),
    engine.save(Athlete(None, "Frank", teams[2])),
]
engine.connection.commit()



In [ ]:
@select_query(Athlete)
def athletes_named_frank():
    f"WHERE {Athlete.name} = 'Frank'"

M, q = athletes_named_frank()
print(q, '\n')

for row in engine.query(M, q).fetchall():
    print(row)

In [ ]:
@select_query(Athlete)
def athletes_on_red_team():
    f"WHERE {Athlete.team.teamname} LIKE 'R%'"

M, q = athletes_on_red_team()
print(q, '\n')

for row in engine.query(M, q).fetchall():
    print(row)

In [ ]:
@select_query(Athlete)
def athletes_in_big_league():
    f"WHERE {Athlete.team.league.leaguename} = 'Big'"
M, q = athletes_in_big_league()
print(q, '\n')

for row in engine.query(M, q).fetchall():
    print(row)

In [ ]:
@select_query(Athlete)
def athletes_in_big_red():
    f"""
    WHERE
        {Athlete.team.league.leaguename} = 'Big'
        AND {Athlete.team.teamname} Like 'R%'
    """
M, q = athletes_in_big_red()
print(q, '\n')

for row in engine.query(M, q).fetchall():
    print(row)

In [ ]:
@select_query(Athlete)
def athletes_in_big_league():
    f"WHERE {Athlete.team.league.leaguename} = 'Big'"

for row in engine.query(*athletes_in_big_league()).fetchall():
    print(row)